## Transfer Learning with Mobilnet V3

Differences between Mobilnet V2 and V3
- Relu activation replaced with wish 
- Layer removal 
- Other optimizations

## Results 
- Half memory usage (~9.5GB to ~ 4.5GB)
- Increased Accuracy 

## Aditional Changes
- Implementing augmentations within imageDataGenerator

## Imports

In [1]:
import os
os.environ['CUDA_DEVICE_ORDER']= "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']= "0"

from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.applications import MobileNetV3Large


In [2]:
from sklearn.model_selection import train_test_split
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
# import os
# # wine = plt.imread('../data/padding/test/class_000/class_000_000.jpg')
# # wine
# len(os.listdir('../data/smallsize/test/'))

564

In [17]:
train_path = '../data/smallclass/train/'
valid_path = '../data/smallclass/val/'
test_path = '../data/smallclass/test/'
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=10,class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(valid_path,target_size=(224,224),batch_size=10,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=10,class_mode='categorical')

Found 18134 images belonging to 73 classes.
Found 6870 images belonging to 73 classes.
Found 6960 images belonging to 73 classes.


In [27]:
base_model = MobileNetV3Small(weights = 'imagenet',include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
x = Dense(128,activation='relu')(x)
preds = Dense(73,activation='softmax')(x)

model=Model(inputs = base_model.input, outputs=preds)

In [28]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_4 (Rescaling)        (None, None, None,   0           ['input_5[0][0]']                
                                3)                                                                
                                                                                                  
 Conv (Conv2D)                  (None, None, None,   432         ['rescaling_4[0][0]']            
                                16)                                                         

 thwiseConv2D)                  72)                              ][0]']                           
                                                                                                  
 expanded_conv_1/depthwise/Batc  (None, None, None,   288        ['expanded_conv_1/depthwise[0][0]
 hNorm (BatchNormalization)     72)                              ']                               
                                                                                                  
 re_lu_132 (ReLU)               (None, None, None,   0           ['expanded_conv_1/depthwise/Batch
                                72)                              Norm[0][0]']                     
                                                                                                  
 expanded_conv_1/project (Conv2  (None, None, None,   1728       ['re_lu_132[0][0]']              
 D)                             24)                                                               
          

                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 96)    0           ['multiply_74[0][0]']            
 /AvgPool (GlobalAveragePooling                                                                   
 2D)                                                                                              
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    2328        ['expanded_conv_3/squeeze_excite/
 /Conv (Conv2D)                                                  AvgPool[0][0]']                  
                                                                                                  
 expanded_conv_3/squeeze_excite  (None, 1, 1, 24)    0           ['expanded_conv_3/squeeze_excite/
 /Relu (ReLU)                                                    Conv[0][0]']                     
          

 tf.__operators__.add_115 (TFOp  (None, 1, 1, 240)   0           ['expanded_conv_4/squeeze_excite/
 Lambda)                                                         Conv_1[0][0]']                   
                                                                                                  
 re_lu_140 (ReLU)               (None, 1, 1, 240)    0           ['tf.__operators__.add_115[0][0]'
                                                                 ]                                
                                                                                                  
 tf.math.multiply_115 (TFOpLamb  (None, 1, 1, 240)   0           ['re_lu_140[0][0]']              
 da)                                                                                              
                                                                                                  
 expanded_conv_4/squeeze_excite  (None, None, None,   0          ['multiply_76[0][0]',            
 /Mul (Mul

 expanded_conv_5/squeeze_excite  (None, None, None,   0          ['multiply_78[0][0]',            
 /Mul (Multiply)                240)                              'tf.math.multiply_118[0][0]']   
                                                                                                  
 expanded_conv_5/project (Conv2  (None, None, None,   9600       ['expanded_conv_5/squeeze_excite/
 D)                             40)                              Mul[0][0]']                      
                                                                                                  
 expanded_conv_5/project/BatchN  (None, None, None,   160        ['expanded_conv_5/project[0][0]']
 orm (BatchNormalization)       40)                                                               
                                                                                                  
 expanded_conv_5/Add (Add)      (None, None, None,   0           ['expanded_conv_4/Add[0][0]',    
          

 )                              144)                             rm[0][0]']                       
                                                                                                  
 expanded_conv_7/expand/BatchNo  (None, None, None,   576        ['expanded_conv_7/expand[0][0]'] 
 rm (BatchNormalization)        144)                                                              
                                                                                                  
 tf.__operators__.add_122 (TFOp  (None, None, None,   0          ['expanded_conv_7/expand/BatchNor
 Lambda)                        144)                             m[0][0]']                        
                                                                                                  
 re_lu_147 (ReLU)               (None, None, None,   0           ['tf.__operators__.add_122[0][0]'
                                144)                             ]                                
          

                                288)                             ]                                
                                                                                                  
 tf.math.multiply_125 (TFOpLamb  (None, None, None,   0          ['re_lu_150[0][0]']              
 da)                            288)                                                              
                                                                                                  
 multiply_83 (Multiply)         (None, None, None,   0           ['expanded_conv_8/expand/BatchNor
                                288)                             m[0][0]',                        
                                                                  'tf.math.multiply_125[0][0]']   
                                                                                                  
 expanded_conv_8/depthwise/pad   (None, None, None,   0          ['multiply_83[0][0]']            
 (ZeroPadd

                                                                                                  
 expanded_conv_9/depthwise/Batc  (None, None, None,   2304       ['expanded_conv_9/depthwise[0][0]
 hNorm (BatchNormalization)     576)                             ']                               
                                                                                                  
 tf.__operators__.add_129 (TFOp  (None, None, None,   0          ['expanded_conv_9/depthwise/Batch
 Lambda)                        576)                             Norm[0][0]']                     
                                                                                                  
 re_lu_154 (ReLU)               (None, None, None,   0           ['tf.__operators__.add_129[0][0]'
                                576)                             ]                                
                                                                                                  
 tf.math.m

                                                                                                  
 tf.math.multiply_132 (TFOpLamb  (None, None, None,   0          ['re_lu_157[0][0]']              
 da)                            576)                                                              
                                                                                                  
 multiply_88 (Multiply)         (None, None, None,   0           ['expanded_conv_10/depthwise/Batc
                                576)                             hNorm[0][0]',                    
                                                                  'tf.math.multiply_132[0][0]']   
                                                                                                  
 expanded_conv_10/squeeze_excit  (None, 1, 1, 576)   0           ['multiply_88[0][0]']            
 e/AvgPool (GlobalAveragePoolin                                                                   
 g2D)     

In [21]:
train_generator.samples

18134

In [ ]:
epochs = 100
optimizer=Adam(learning_rate=.0001)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.fit(train_generator, validation_data=valid_generator, epochs=epochs,
          steps_per_epoch=train_generator.samples//train_generator.batch_size,
         validation_steps=valid_generator.samples//valid_generator.batch_size)

Epoch 1/100
1813/1813 [==============================] - 94s 50ms/step - loss: 2.8025 - accuracy: 0.3191 - val_loss: 5.5851 - val_accuracy: 0.0132
Epoch 2/100
 179/1813 [=>............................] - ETA: 1:01 - loss: 1.2816 - accuracy: 0.6480